In [1]:
import os
import getpass

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain_chroma import Chroma

from datetime import datetime, timedelta
import requests

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GENERATIVE_LANGUAGE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
WEATHER_API = os.getenv('WEATHER_API')

In [3]:
def get_date_for_day(day_name):
    """
    Função para construir a data para um dia da semana específico.
    """
    today = datetime.today()
    today_weekday = today.weekday()

    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    try:
        requested_weekday = days_of_week.index(day_name)
    except ValueError:
        return None

    if requested_weekday == today_weekday:
       return today.strftime("%Y-%m-%d")
    
    days_until_requested = (requested_weekday - today_weekday + 7) % 7
    target_date = today + timedelta(days=days_until_requested)
    print(target_date)
    return target_date.strftime("%Y-%m-%d")

In [ ]:
import requests

BASE_URL = "http://api.weatherapi.com/v1/forecast.json"

def weatherapi_forecast_periods(date_string: str) -> str:
    """
    Obtém a previsão do tempo para a cidade de Natal em uma data específica,
    separada em manhã, tarde e noite.

    Args:
        date_string (str): Uma string contendo a data no formato yyyy-mm-dd.

    Returns:
        str: Uma string contendo as previsões separadas por períodos para a data especificada.
    """
    try:
        params = {
            "key": WEATHER_API,
            "q": "Natal",
            "dt": date_string,
            "aqi": "no",
            "alerts": "no",
            "lang": "pt"
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if data and data.get("forecast") and data["forecast"].get("forecastday"):
            forecast_data = data["forecast"]["forecastday"][0]
            hourly_data = forecast_data.get("hour", [])

            periods = {
                "Manhã": range(5, 12),
                "Tarde": range(13, 18),
                "Noite": range(18, 24)
            }

            result = f"Previsão para {date_string} em Natal:\n"
            for period, hours in periods.items():
                result += f"\n{period}:\n"
                filtered_data = [
                    {
                        "hora": hour["time"].split(" ")[1],
                        "temperatura": hour["temp_c"],
                        "condicao": hour["condition"]["text"],
                        "chance_de_chuva": hour["chance_of_rain"],
                        "umidade": hour["humidity"]
                    }
                    for hour in hourly_data if int(hour["time"].split(" ")[1].split(":")[0]) in hours
                ]
                if filtered_data:
                    for hour in filtered_data:
                        result += (
                            f"  Hora: {hour['hora']} - "
                            f"Temp: {hour['temperatura']}°C, "
                            f"Condição: {hour['condicao']}, "
                            f"Chance de chuva: {hour['chance_de_chuva']}%, "
                            f"Umidade: {hour['umidade']}%\n"
                        )
                else:
                    result += "  Nenhuma previsão encontrada para este período.\n"

            return result
        else:
            return "Previsão não encontrada para essa data."
    except requests.exceptions.RequestException as e:
        return f"Erro ao buscar informações meteorológicas: {str(e)}"
    except Exception as e:
        return f"Erro inesperado: {str(e)}"


In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [7]:
from get_embedding_function import get_embedding_function
CHROMA_PATH = "chroma"

def query_rag(query_text):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text

def query_rag_tool(query_text: str) -> str:
    return query_rag(query_text)

In [8]:
ddg_search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Use esta ferramenta para buscar informações recentes e responda ao usuário com os resultados encontrados. É importante que a resposta contenha o que foi recuperado da pesquisa."
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="Você deve utilizar essa ferramenta quando o usuário solicitar informações sobre o clima. Se o usuário pedir informações para um período de tempo (por exemplo, 'de sexta a domingo'), você DEVE usar a ferramenta separadamente para cada dia desse período e depois combinar os resultados na resposta final. Se o usuário fornecer apenas uma data, use a ferramenta apenas uma vez."
    ),
    Tool(
        name="Query RAG",
        func=query_rag_tool,
        description="A perguntas relacionadas ao turismo da cidade de Natal no Rio Grande do Norte. Se as informações não forem suficientes no Query RAG para responder ao usuário utilize a ferramenta do DuckDuckGo Search"
    )
]

In [9]:
agent_prompt = hub.pull("tales/agente_turismo")

In [10]:
print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'data_atual', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_turismo', 'lc_hub_commit_hash': '01a133cdf82e821cdb37a291a75b2f86c7d0327b475cc84015bb3ca21cba5167'} template='Seu nome é Sofia. Você é uma assistente pessoal de inteligência artificial de classe mundial, especializada em viagens e turismo. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações, planejar roteiros personalizados e resolver problemas relacionados a viagens. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática) com foco em planejamento de viagens.\n\n"Sofia" foi projetada para auxiliar usuários em todas as etapas de suas aventuras, desde a escolha do destino até a criação de itinerários detalhados e dicas práticas. Como modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas informações fornecidas pelos usuá

In [11]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [12]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=5, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_26712\582812787.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=5, chat_memory=history, memory_key="chat_history")


In [13]:
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [14]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [15]:
print(data_atual)

sábado, 2024/12/28


In [16]:
response = agent_executor.invoke({"input": f"""
                                  Boa noite
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Boa noite!  Qual cidade você pretende visitar?


> Finished chain.
Boa noite!  Qual cidade você pretende visitar?


In [17]:
response = agent_executor.invoke({"input": f"""
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma cidade linda.  Quantos dias você planeja passar lá, e quais os dias da semana?
```


> Finished chain.
Ótimo! Natal é uma cidade linda.  Quantos dias você planeja passar lá, e quais os dias da semana?
```


In [18]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo passar da segunda dia 30 até a quarta
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! De segunda-feira, dia 30, até quarta-feira.  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


> Finished chain.
Perfeito! De segunda-feira, dia 30, até quarta-feira.  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


In [19]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo vizitar praias e restaurantes regionais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Entendido! Praias e restaurantes regionais.  Você tem um orçamento aproximado ou um limite de gastos para essa viagem?
```


> Finished chain.
Entendido! Praias e restaurantes regionais.  Você tem um orçamento aproximado ou um limite de gastos para essa viagem?
```


In [20]:
response = agent_executor.invoke({"input": f"""
                                  Sem incluir Hóteis pretendo gastar no máximo 1000 reais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30Previsão para 2024-12-30 em Natal:

Manhã:
  Hora: 05:00 - Temp: 26.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 79%
  Hora: 06:00 - Temp: 27.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 75%, Umidade: 73%
  Hora: 07:00 - Temp: 28.5°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 69%
  Hora: 08:00 - Temp: 29.1°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 66%
  Hora: 09:00 - Temp: 30.0°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 84%, Umidade: 62%
  Hora: 10:00 - Temp: 30.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 62%, Umidade: 60%
  Hora: 11:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 58%

Tarde:
  Hora: 13:00 - Temp: 30.6°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 55%
  Hora: 14:00 - T

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-31Previsão para 2024-12-31 em Natal:

Manhã:
  Hora: 05:00 - Temp: 27.1°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 67%
  Hora: 06:00 - Temp: 27.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 65%
  Hora: 07:00 - Temp: 29.0°C, Condição: Sol, Chance de chuva: 0%, Umidade: 60%
  Hora: 08:00 - Temp: 29.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 58%
  Hora: 09:00 - Temp: 30.6°C, Condição: Sol, Chance de chuva: 0%, Umidade: 56%
  Hora: 10:00 - Temp: 30.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 11:00 - Temp: 30.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%

Tarde:
  Hora: 13:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 14:00 - Temp: 30.2°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 15:00 - Temp: 29.5°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 16:00 - Temp: 28.4°C, Condição: So

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025-01-01Previsão para 2025-01-01 em Natal:

Manhã:
  Hora: 05:00 - Temp: 26.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 83%, Umidade: 75%
  Hora: 06:00 - Temp: 27.7°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 69%
  Hora: 07:00 - Temp: 28.9°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 62%
  Hora: 08:00 - Temp: 29.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 59%
  Hora: 09:00 - Temp: 30.4°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 10:00 - Temp: 30.9°C, Condição: Sol, Chance de chuva: 0%, Umidade: 55%
  Hora: 11:00 - Temp: 31.1°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%

Tarde:
  Hora: 13:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 14:00 - Temp: 30.1°C, Condição: Sol, Chance de chuva: 0%, Umidade: 56%
  Hora: 15:00 - Temp: 29.3°C, Condição: Sol, Chance de chuva: 0%, Umidade:

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com um orçamento de R$1000,00 (excluindo hotéis), podemos planejar um roteiro incrível em Natal focado em praias e restaurantes regionais.  Considerando as previsões do tempo para os dias 30/12 (segunda), 31/12 (terça) e 01/01 (quarta), vamos criar um roteiro personalizado.  Para começar, preciso de mais algumas informações: você prefere praias mais agitadas com estrutura completa ou praias mais tranquilas e reservadas?
```


> Finished chain.
Ótimo! Com um orçamento de R$1000,00 (excluindo hotéis), podemos planejar um roteiro incrível em Natal focado em praias e restaurantes regionais.  Considerando as previsões do tempo para os dias 30/12 (segunda), 31/12 (terça) e 01/01 (quarta), vamos criar um roteiro personalizado.  Para começar, preciso de mais algumas informações: você prefere praias mais agitadas com estrutura completa ou praias mais tranquilas e reservadas?
```


In [21]:
response = agent_executor.invoke({"input": f"""
                                  Eu quero aproveitar uma praia por dia e relaxar
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30Previsão para 2024-12-30 em Natal:

Manhã:
  Hora: 05:00 - Temp: 26.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 79%
  Hora: 06:00 - Temp: 27.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 75%, Umidade: 73%
  Hora: 07:00 - Temp: 28.5°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 69%
  Hora: 08:00 - Temp: 29.1°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 66%
  Hora: 09:00 - Temp: 30.0°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 84%, Umidade: 62%
  Hora: 10:00 - Temp: 30.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 62%, Umidade: 60%
  Hora: 11:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 58%

Tarde:
  Hora: 13:00 - Temp: 30.6°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 55%
  Hora: 14:00 - T

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-31Previsão para 2024-12-31 em Natal:

Manhã:
  Hora: 05:00 - Temp: 27.1°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 67%
  Hora: 06:00 - Temp: 27.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 65%
  Hora: 07:00 - Temp: 29.0°C, Condição: Sol, Chance de chuva: 0%, Umidade: 60%
  Hora: 08:00 - Temp: 29.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 58%
  Hora: 09:00 - Temp: 30.6°C, Condição: Sol, Chance de chuva: 0%, Umidade: 56%
  Hora: 10:00 - Temp: 30.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 11:00 - Temp: 30.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%

Tarde:
  Hora: 13:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 14:00 - Temp: 30.2°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 15:00 - Temp: 29.5°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 16:00 - Temp: 28.4°C, Condição: So

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025-01-01Previsão para 2025-01-01 em Natal:

Manhã:
  Hora: 05:00 - Temp: 26.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 83%, Umidade: 75%
  Hora: 06:00 - Temp: 27.7°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 69%
  Hora: 07:00 - Temp: 28.9°C, Condição: Parcialmente nublado, Chance de chuva: 0%, Umidade: 62%
  Hora: 08:00 - Temp: 29.8°C, Condição: Sol, Chance de chuva: 0%, Umidade: 59%
  Hora: 09:00 - Temp: 30.4°C, Condição: Sol, Chance de chuva: 0%, Umidade: 57%
  Hora: 10:00 - Temp: 30.9°C, Condição: Sol, Chance de chuva: 0%, Umidade: 55%
  Hora: 11:00 - Temp: 31.1°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%

Tarde:
  Hora: 13:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 54%
  Hora: 14:00 - Temp: 30.1°C, Condição: Sol, Chance de chuva: 0%, Umidade: 56%
  Hora: 15:00 - Temp: 29.3°C, Condição: Sol, Chance de chuva: 0%, Umidade:

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com base nas suas preferências e no seu orçamento, e considerando as previsões do tempo, sugiro o seguinte roteiro para sua viagem a Natal:

**Segunda-feira, 30 de Dezembro de 2024:**

* **Manhã:**  Como a previsão indica chuva pela manhã, sugiro começar o dia mais tarde.  Após o café da manhã no seu hotel,  aproveite a tarde para visitar a Praia de Ponta Negra.  Apesar da possibilidade de chuva pela manhã, a tarde costuma ser ensolarada em Natal.  Ponta Negra oferece boa estrutura, quiosques e restaurantes.
* **Tarde:** Relaxe na Praia de Ponta Negra, aproveite o sol e dê um mergulho.  Almoce em um dos quiosques da praia, provando frutos do mar frescos.
* **Noite:**  Jante em um restaurante regional em Ponta Negra.  Existem muitas opções com preços acessíveis, oferecendo a culinária potiguar.  Procure por restaurantes que sirvam pratos típicos como carne de sol, baião de dois e camarão.


**Terça-feira, 31 de Dezem

In [22]:
response = agent_executor.invoke({"input": f"""
                                  Obrigado
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: De nada! Tenha uma ótima viagem para Natal!  Espero que aproveite ao máximo as praias e os restaurantes regionais.  Se precisar de algo mais, não hesite em me perguntar.
```


> Finished chain.
De nada! Tenha uma ótima viagem para Natal!  Espero que aproveite ao máximo as praias e os restaurantes regionais.  Se precisar de algo mais, não hesite em me perguntar.
```


In [23]:
print(response["chat_history"])

Human: 
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  
AI: Ótimo! Natal é uma cidade linda.  Quantos dias você planeja passar lá, e quais os dias da semana?
```
Human: 
                                  Eu planejo passar da segunda dia 30 até a quarta
                                  
AI: Perfeito! De segunda-feira, dia 30, até quarta-feira.  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```
Human: 
                                  Eu planejo vizitar praias e restaurantes regionais
                                  
AI: Entendido! Praias e restaurantes regionais.  Você tem um orçamento aproximado ou um limite de gastos para essa viagem?
```
Human: 
                                  Sem incluir Hóteis pretendo gastar no máximo 1000 reais
                                  
AI: Ótimo! Com um orçamento